In [11]:
!mkdir data
!wget -O ./data/Jonh_Wick_small.mp4 https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true

mkdir: cannot create directory ‘data’: File exists
--2021-07-04 15:00:41--  https://github.com/chulminkw/DLCV/blob/master/data/video/John_Wick_small.mp4?raw=true
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4 [following]
--2021-07-04 15:00:41--  https://github.com/chulminkw/DLCV/raw/master/data/video/John_Wick_small.mp4
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4 [following]
--2021-07-04 15:00:41--  https://raw.githubusercontent.com/chulminkw/DLCV/master/data/video/John_Wick_small.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubuserc

In [12]:
!mkdir ./pretrained
!wget -O ./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
!wget -O ./pretrained/config_graph.pbtxt https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt

mkdir: cannot create directory ‘./pretrained’: File exists
--2021-07-04 15:00:43--  http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.122.128, 2607:f8b0:4004:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.122.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381355771 (364M) [application/x-tar]
Saving to: ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’

./pretrained/faster 100%[===================>] 363.69M   235MB/s    in 1.5s    

2021-07-04 15:00:45 (235 MB/s) - ‘./pretrained/faster_rcnn_resnet50_coco_2018_01_28.tar.gz’ saved [381355771/381355771]

--2021-07-04 15:00:45--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.

In [23]:
!tar -xvf ./pretrained/faster*.tar.gz -C ./pretrained 

faster_rcnn_resnet50_coco_2018_01_28/
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.index
faster_rcnn_resnet50_coco_2018_01_28/checkpoint
faster_rcnn_resnet50_coco_2018_01_28/pipeline.config
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.data-00000-of-00001
faster_rcnn_resnet50_coco_2018_01_28/model.ckpt.meta
faster_rcnn_resnet50_coco_2018_01_28/saved_model/
faster_rcnn_resnet50_coco_2018_01_28/saved_model/saved_model.pb
faster_rcnn_resnet50_coco_2018_01_28/saved_model/variables/
faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb


In [13]:
import cv2
import matplotlib.pyplot as plt
import time 

%matplotlib inline

In [24]:
cv_net = cv2.dnn.readNetFromTensorflow('./pretrained/faster_rcnn_resnet50_coco_2018_01_28/frozen_inference_graph.pb', 
                                     './pretrained/config_graph.pbtxt')

In [14]:
# OpenCV Tensorflow Faster-RCNN용
labels_to_names_0 = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                    10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                    20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                    30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                    40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                    50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                    60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                    70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                    80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                    90:'hair brush'}

In [28]:
def get_detected_img(cv_net, img_array, score_threshold, use_copied_array=True, is_print=True):
  
  green_color = (0, 255, 0)
  red_color = (0, 0, 255)
  
  rows = img_array.shape[0]
  cols = img_array.shape[1]

  draw_img = None
  if use_copied_array:
    draw_img = img_array.copy()
  else:
    draw_img = img_array


  cv_net.setInput(cv2.dnn.blobFromImage(img_array, swapRB=True, crop=False))
  start = time.time()
  cv_out = cv_net.forward()


  for detection in cv_out[0,0,:,:]:
    score = float(detection[2])
    class_id = int(detection[1])

    if score > score_threshold:
      x1 = int(detection[3] * cols)
      y1 = int(detection[4] * rows)
      x2 = int(detection[5] * cols)
      y2 = int(detection[6] * rows)

      text = "{} : {:.4f}".format(labels_to_names_0[class_id], score)

      cv2.rectangle(draw_img, (x1,y1), (x2,y2), color=green_color, thickness=2)
      cv2.putText(draw_img, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.4, red_color, 1)
  
  if is_print:
    print("Detection 수행시간 : {}초".format(round(time.time() - start,2)))
  
  return draw_img

### Video Capture과 Video Writer 설정하기

In [29]:
video_input_path = '/content/data/Jonh_Wick_small.mp4'

cap = cv2.VideoCapture(video_input_path)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("총 Frame의 갯수 : {}".format(frame_count))

총 Frame의 갯수 : 58


In [33]:
video_input_path = '/content/data/Jonh_Wick_small.mp4'
video_output_path = './data/John_Wick_small_cv01.mp4'

cap = cv2.VideoCapture(video_input_path)

codec = cv2.VideoWriter_fourcc(*'XVID')

vid_fps = cap.get(cv2.CAP_PROP_FPS)
vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("총 Frame의 갯수 : {}".format(frame_count))
print("FPS : {}".format(vid_fps))
print("VIDEO SIZE : {}".format(vid_size))


총 Frame의 갯수 : 58
FPS : 30.00051725029742
VIDEO SIZE : (1072, 460)


In [34]:
# Frame별로 Object Detection 수행
while True:

  hasFrame, img_frame = cap.read()
  if not hasFrame:
    print("더 이상 처리할 frame이 없습니다.")
    break
  
  img_frame = get_detected_img(cv_net, img_frame, score_threshold=0.5, use_copied_array=False, is_print=True)
  vid_writer.write(img_frame)

vid_writer.release()
cap.release()

Detection 수행시간 : 7.71초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.56초
Detection 수행시간 : 7.6초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.58초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.54초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.57초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.52초
Detection 수행시간 : 7.53초
Detection 수행시간 : 7.53초
Detection 수행시간 : 7.53초
Detection 수행시간 : 7.61초
Detection 수행시간 : 7.58초
Detection 수행시간 : 7.62초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.53초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.56초
Detection 수행시간 : 7.56초
Detection 수행시간 : 7.54초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.53초
Detection 수행시간 : 7.59초
Detection 수행시간 : 7.56초
Detection 수행시간 : 7.55초
Detection 수행시간 : 7.57초
Detection 수행시간 : 7.52초
Detection 수행시간 : 7.61초
Detection 수행시간 : 7.63초
Detection 수행시간 : 7.78초
Detection 수행시간 : 7.74초
Detection 수행시간 : 7.75초
Detection 수행시간 : 7.74초
Detection 수행시간 : 7.65초
Detection 수행시간 : 7.77초
Detection 수행시간 : 7.76초
Detection 수행시간 : 7.66초
Detection 수행

In [ ]:
input_path = '/content/data/Jonh_Wick_small.mp4'
output_path = '/content/data/Jonh_Wick_small_cv1.mp4'

cap = cv2.VideoCapture(input_path)
codec = cv2.VideoWriter_fourcc(output_path)

vid_size = (round(cap.get(cv2.CAP_PORP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PORP_FRAME_HEIGHT)))
vid_fps = cap.get(cv2.CAP_PORP_FRAM_FPS)

vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size)


green_color = (0,255,0)
red_color = (0,0,255)

while True:

  hasFrame, img_frame = cap.read()
  if not hasFrame:
    print("END")
    break

    rows = img_frame.shape[0]
    cols = img_frame.shape[1]

    cv_out = cv_net.forward()
    start = time.time()

    for detection in cv_out[0,0,:,:]:
      score = detection[2]
      class_id = labels_to_names_0[int(detection[1])]

      if score>0.5:
        x1 = int(detection[3]) * cols
        y1 = int(detection[4]) * rows
        x2 = int(detection[5]) * cols
        y2 = int(detection[6]) * rows

        text = "{} : {.:4f}".format(class_id, score)
        cv2.rectangle(img_frame, (x1,y1), (x2,y2), color=green_color, thickness=2)
        cv2.putText(img_frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
  print('Detection 수행 시간:', round(time.time()-start, 2),'초')
  vid_writer.write(img_frame)

vid_writer.release()
cap.release()


In [ ]:
def do_detected_video(cv_net, input_path, output_path, score_threshold, is_print):
    
    cap = cv2.VideoCapture(input_path)

    codec = cv2.VideoWriter_fourcc(*'XVID')

    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS)

    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size) 

    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('총 Frame 갯수:', frame_cnt)

    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    while True:
        hasFrame, img_frame = cap.read()
        if not hasFrame:
            print('더 이상 처리할 frame이 없습니다.')
            break
        
        img_frame = get_detected_img(cv_net, img_frame, score_threshold=score_threshold, use_copied_array=False, is_print=is_print)
        
        vid_writer.write(img_frame)
    # end of while loop

    vid_writer.release()
    cap.release()

In [ ]:
do_detected_video(cv_net, '/content/data/Jonh_Wick_small.mp4', './data/John_Wick_small_02.mp4', 0.2, False)